In [10]:
import numpy as np
import pandas as pd

## Load Dataset: Beijing PM2.5

In [11]:
dataset_root = "../datasets/"
csvfile = os.path.join(dataset_root, "beijing_pm2.5", "PRSA_data_2010.1.1-2014.12.31.csv")
df = pd.read_csv(csvfile)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43824 entries, 0 to 43823
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   No      43824 non-null  int64  
 1   year    43824 non-null  int64  
 2   month   43824 non-null  int64  
 3   day     43824 non-null  int64  
 4   hour    43824 non-null  int64  
 5   pm2.5   41757 non-null  float64
 6   DEWP    43824 non-null  int64  
 7   TEMP    43824 non-null  float64
 8   PRES    43824 non-null  float64
 9   cbwd    43824 non-null  object 
 10  Iws     43824 non-null  float64
 11  Is      43824 non-null  int64  
 12  Ir      43824 non-null  int64  
dtypes: float64(4), int64(8), object(1)
memory usage: 4.3+ MB


## Feature Definitions

List of plants to grow, with scores that rate their performance in a given interval of time. Scores should be floating-point numbers within a range (possibly from `[0,10]`) for each plant. The score matrix should have columns that correspond to each plant, and rows that correspond to each time interval.

For example:
```python
features = ['tomato', 'sunflower', 'cucumber']
scores = [[3, 7, 10], [2, 1, 5], [7, 9, 11]]
```

In [15]:
features = ['tomato', 'sunflower', 'cucumber']
scores = np.random.uniform(0.0, 10.0, size=(df.shape[0],len(features))) # Random matrix of scores between [0,10]

In [16]:
scores

array([[7.77168866, 2.18008585, 7.12964378],
       [7.66446765, 3.01934004, 2.99664256],
       [6.71976144, 7.750026  , 0.43848982],
       ...,
       [3.30067684, 6.81362248, 6.6527205 ],
       [0.44770491, 1.02317399, 9.40958871],
       [4.052097  , 8.54071029, 6.22722581]])